## Date parsing automation in Pandas

In [1]:
import numpy as np
import pandas as pd
from itertools import product,permutations,combinations
import re

pd.options.display.max_columns=999

The logic behind this class is that:
Python doesn't parse dates very accurately, so i decided to use regex to make the automation better than python's built-in functions. 
Length of all combinations and permutations of date in python is 36 which has been calculated in init of the GetDateDiff class. 

I've created a small dataset considering the worst ways of collecting data.

Most columns which might have date in them are of type 'object'. so we can through all the object features in the dataset and take a sample of rows (sample_size argument can be passed to the function main. default 15). Next part is bruteforcing all the formats against the sample rows which gives us all the different formats found in that column.(varies on sample_size). If multiple formats are found then each row of the feature has to be transformed one by one.

Note: This algorithm is not at its most efficient state and can be improved. I'll continue to work on it to make it better as a personal project.

In [2]:
class GetDateDiff:
  
    def __init__(self):
    # Initializing regex for all date formats in python using permutation and combination 
        self.reg={"%d":"(0[1-9]|[12]\d|3[01])","%y":"([0-9]{2})","%Y":"([0-9]{4})","%m":"(0[1-9]|1\d)","%b":"[A-Za-z]{3}",
         "%B":"[A-Za-z]{4,10}"}
        day=["%d"]
        month=["%B","%b","%m"]
        year=["%Y","%y"]
        products=[i for i in product(day,month,year)]
        self.formats={} # all formats as keys and their regex as values
        for i in products:
            for j in permutations(i):
                self.formats["".join(j)]=self.reg[j[0]]+self.reg[j[1]]+self.reg[j[2]]

  
    def clean_column(self,col):
        #this function removes all - . / and white spaces from column values
        return col.apply(lambda x: re.sub("[-./ ]+","",str(x)))

  
  
    def get_formats(self,sample):
        #this function is used to get different formats from a sample of dataframe.
        forms=set()
        sample=self.clean_column(sample)
        for i in sample:
            for form in self.formats:
                try:
                    pd.to_datetime(i,format=form,exact=False)
                    pattern = re.compile(self.formats[form])
                    if pattern.match(i) is not None:
                        forms.add(form)
                    break
                except:
                    pass
        result=list(forms)
        result.sort()
        return result

    def convert_all(self,col,col_formats):
        #this function is used to convert values of columns with multiple date formats.
        converted=[]
        for i in range(len(col)):
            for f in self.formats:
                try:
                    pattern = re.compile(self.formats[f])
                    if pattern.match(col[i]) is not None:
                        row=pd.to_datetime(pd.Series(col[i]),format=f,exact=False)
                        converted.append(row[0])
                        break
                except:
                    continue          
        return pd.to_datetime(converted)

    def main(self,df,sample_size=15):
        #date_cols contains name of all columns which have dates
        date_cols=[]
        # nof contains keys of date columns and formats found in them after get_formats is called
        nof={}
        for col in df.columns:
            if df[col].dtype=='object':
                #take sample_size random values from column
                sample=df[col].sample(sample_size)
                #save all formats as list in nof with key as column name
                nof[col]=self.get_formats(sample)
                # remove all those columns which did not have any date ie. which are not date columns
                if len(nof[col])==0:
                    del nof[col]
                    
        for col,formats in nof.items():
            if len(formats)==1:
                #if a columns has only one date format. simply convert the column
                try:
                    df[col]=pd.to_datetime(self.clean_column(df[col]),format=formats[0],exact=False)
                    date_cols.append(col)
                except:
                    print("Error for",col,"format:",formats[0])

            else:
              # else bruteforce all formats again each value and convert the columns
                temp=self.convert_all(self.clean_column(df[col]),formats)
                df[col]=temp
                date_cols.append(col)
        #combinations of date columns to find difference of date between them
        for combination in combinations(date_cols,2):
            df[combination[0]+"_"+combination[1]]=df[combination[0]]-df[combination[1]]

        return df        
  


In [3]:
q1=pd.read_csv('/content/drive/My Drive/custom.csv')
q1.head()

,date1,date3,date4,date5,date6,date7
0,12Jan2019,12-01-2019,12/01/19,12.01.2019,Jan122019,Jan122019
1,13feb2018,13-02-2018,13/02/18,13.02.2018,13feb 2018,201813feb
2,31mar2014,31-03-2014,31/03/14,31.03.2014,31 mar2014,mar2014 31
3,21apr2012,21-04-2012,21/04/12,21.04.2012,21 apr2012,21 apr2012
4,10oct2020,10-10-2020,10/10/20,10.10.2020,10oct 2020,10oct 2020


In [4]:
getdiff=GetDateDiff()
q1=getdiff.main(q1)

In [5]:
q1.head()

,date1,date3,date4,date5,date6,date7,date1_date3,date1_date4,date1_date5,date1_date6,date1_date7,date3_date4,date3_date5,date3_date6,date3_date7,date4_date5,date4_date6,date4_date7,date5_date6,date5_date7,date6_date7
0,2019-01-12,2019-01-12,2019-01-12,2019-01-12,2019-01-12,2019-01-12,0 days,0 days,0 days,0 days,0 days,0 days,0 days,0 days,0 days,0 days,0 days,0 days,0 days,0 days,0 days
1,2018-02-13,2018-02-13,2018-02-13,2018-02-13,2018-02-13,1813-02-20,0 days,0 days,0 days,0 days,74868 days,0 days,0 days,0 days,74868 days,0 days,0 days,74868 days,0 days,74868 days,74868 days
2,2014-03-31,2014-03-31,2014-03-31,2014-03-31,2014-03-31,2014-03-31,0 days,0 days,0 days,0 days,0 days,0 days,0 days,0 days,0 days,0 days,0 days,0 days,0 days,0 days,0 days
3,2012-04-21,2012-04-21,2012-04-21,2012-04-21,2012-04-21,2012-04-21,0 days,0 days,0 days,0 days,0 days,0 days,0 days,0 days,0 days,0 days,0 days,0 days,0 days,0 days,0 days
4,2020-10-10,2020-10-10,2020-10-10,2020-10-10,2020-10-10,2020-10-10,0 days,0 days,0 days,0 days,0 days,0 days,0 days,0 days,0 days,0 days,0 days,0 days,0 days,0 days,0 days
